<a href="https://colab.research.google.com/github/Monishgalla/Classifying-different-land-uses-in-satellite-images-using-CNN/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive')


data_dir = '/content/drive/MyDrive/dataset/UCMerced_LandUse/Images'  # Directory containing the 21 folders
img_width, img_height = 150, 150  # Desired image dimensions

# List all the class labels
class_labels = os.listdir(data_dir)

# Initialize empty lists to store the images and labels
images = []
labels = []

# Iterate through each folder
for class_label in class_labels:
    folder_path = os.path.join(data_dir, class_label)

    if os.path.isdir(folder_path):
        image_files = os.listdir(folder_path)

        # Load and resize the images, and store the labels
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            img = cv2.imread(image_path)
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(class_label)

# Convert the images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Perform label encoding on the class labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Perform one-hot encoding on the labels
num_classes = len(class_labels)
labels = to_categorical(labels, num_classes=num_classes)

# Shuffle the data
indices = np.arange(len(images))
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# Perform data normalization
images = images.astype('float32') / 255.0

# Split the data into training, validation, and testing sets
train_split = 0.7  # Percentage of data for training
val_split = 0.15  # Percentage of data for validation
test_split = 0.15  # Percentage of data for testing

num_samples = len(images)
num_train_samples = int(train_split * num_samples)
num_val_samples = int(val_split * num_samples)

x_train = images[:num_train_samples]
y_train = labels[:num_train_samples]

x_val = images[num_train_samples:num_train_samples + num_val_samples]
y_val = labels[num_train_samples:num_train_samples + num_val_samples]

x_test = images[num_train_samples + num_val_samples:]
y_test = labels[num_train_samples + num_val_samples:]

print("Data preprocessing completed.")
print("Number of training samples:", len(x_train))
print("Number of validation samples:", len(x_val))
print("Number of testing samples:", len(x_test))

Mounted at /content/drive
Data preprocessing completed.
Number of training samples: 1482
Number of validation samples: 317
Number of testing samples: 319


In [6]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

data_dir = '/content/drive/MyDrive/dataset/UCMerced_LandUse/Images'  # Directory containing the 21 folders
img_width, img_height = 150, 150  # Desired image dimensions
num_classes = 21  # Number of land use classes

# List all the class labels
class_labels = os.listdir(data_dir)

# Initialize empty lists to store the images and labels
images = []
labels = []

# Iterate through each folder
for class_label in class_labels:
    folder_path = os.path.join(data_dir, class_label)

    if os.path.isdir(folder_path):
        image_files = os.listdir(folder_path)

        # Load and resize the images, and store the labels
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            img = cv2.imread(image_path)
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(class_label)

# Convert the images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Perform label encoding on the class labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Perform one-hot encoding on the labels
labels = to_categorical(labels, num_classes=num_classes)

# Perform data normalization
images = images.astype('float32') / 255.0

# Split the data into training, validation, and testing sets
train_split = 0.7  # Percentage of data for training
val_split = 0.15  # Percentage of data for validation
test_split = 0.15  # Percentage of data for testing

num_samples = len(images)
num_train_samples = int(train_split * num_samples)
num_val_samples = int(val_split * num_samples)

x_train = images[:num_train_samples]
y_train = labels[:num_train_samples]

x_val = images[num_train_samples:num_train_samples + num_val_samples]
y_val = labels[num_train_samples:num_train_samples + num_val_samples]

x_test = images[num_train_samples + num_val_samples:]
y_test = labels[num_train_samples + num_val_samples:]

In [7]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [8]:
# Load the pre-trained VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers of the VGG16 model
for layer in vgg_model.layers:
    layer.trainable = False


58889256/58889256 [==============================] - 0s 0us/step


In [9]:
# Create a new model by combining the VGG16 model and the CNN model
combined_model = Sequential()
combined_model.add(vgg_model)
combined_model.add(Flatten())
combined_model.add(Dense(128, activation='relu'))
combined_model.add(Dropout(0.5))
combined_model.add(Dense(num_classes, activation='softmax'))

# Compile the combined model
combined_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the combined model
combined_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model on the test set
loss, accuracy = combined_model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/10
47/47 [==============================] - 571s 12s/step - loss: 2.4592 - accuracy: 0.2848 - val_loss: 4.0040 - val_accuracy: 0.0063
Epoch 2/10
47/47 [==============================] - 552s 12s/step - loss: 1.4892 - accuracy: 0.6053 - val_loss: 4.3360 - val_accuracy: 0.0032
Epoch 3/10
47/47 [==============================] - 497s 11s/step - loss: 1.0727 - accuracy: 0.7045 - val_loss: 4.6938 - val_accuracy: 0.0095
Epoch 4/10
47/47 [==============================] - 555s 12s/step - loss: 0.8297 - accuracy: 0.7645 - val_loss: 5.0294 - val_accuracy: 0.0000e+00
Epoch 5/10
47/47 [==============================] - 553s 12s/step - loss: 0.6947 - accuracy: 0.8151 - val_loss: 5.1001 - val_accuracy: 0.0284
Epoch 6/10
47/47 [==============================] - 553s 12s/step - loss: 0.6129 - accuracy: 0.8421 - val_loss: 5.2653 - val_accuracy: 0.0221
Epoch 7/10
47/47 [==============================] - 559s 12s/step - loss: 0.5222 - accuracy: 0.8603 - val_loss: 5.4397 - val_accuracy: 0.0379
Ep